In [1]:
import numpy as np
import pandas as pd
import json
from geopy.geocoders import Nominatim
from geopy.geocoders import Nominatim
import geocoder
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
print('Libraries imported.')

Libraries imported.


In [1]:
data = requests.get("https://en.wikipedia.org/wiki/Towns_in_Karachi").text
soup = BeautifulSoup(data, 'html.parser')
neighborhoodList = []
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})
def get_latlng(neighborhood):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Mumbai, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords
coords = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']
kl_df.head()


NameError: name 'requests' is not defined

In [2]:
address = 'Karachi, Pakistan'
geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Karachi {}, {}.'.format(latitude, longitude))

NameError: name 'Nominatim' is not defined

In [8]:
CLIENT_ID = 'J2HRL5UW01505JZ1TIF0T45AYZYHMXGEB2VXHRTOC3XT1OVV' # your Foursquare ID
CLIENT_SECRET = 'DOA1LVGX0RDHXLZOJPMEHRHRWR2Z25VKMLGMJ2MRSGMYE0QO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: J2HRL5UW01505JZ1TIF0T45AYZYHMXGEB2VXHRTOC3XT1OVV
CLIENT_SECRET:DOA1LVGX0RDHXLZOJPMEHRHRWR2Z25VKMLGMJ2MRSGMYE0QO


In [3]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

venues_df = pd.DataFrame(venues)
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
venues_df.head()

NameError: name 'kl_df' is not defined

In [4]:
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()
kl_grouped.head()

NameError: name 'pd' is not defined

In [5]:
kl_mall = kl_grouped[["Neighborhoods","Shopping Mall"]]
kl_mall.head()

NameError: name 'kl_grouped' is not defined

In [6]:
kclusters = 3
kl_clustering = kl_mall.drop(["Neighborhoods"], 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)
kl_merged = kl_mall.copy()
kl_merged["Cluster Labels"] = kmeans.labels_
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged = kl_merged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged.head() 

NameError: name 'kl_mall' is not defined

In [14]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [7]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

NameError: name 'kl_merged' is not defined

In [8]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

NameError: name 'kl_merged' is not defined

In [9]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

NameError: name 'kl_merged' is not defined

In [18]:
map_clusters.save('map_clusters.html')